__Задача__
- Загрузить уже собранные записи об изделиях, а также их сроков изготовления (`./data/crm/train.json`)
- Запросить расчеты временем из api калькулятора
- Полученные данные записать в файл для дальнейшего сравнения `./data/calc/prod_t.json`

То есть сравнить фактическое время производства с просчетом

In [ ]:
import re
import os
import json
import time
import requests
import datetime
import pandas as pd

from IPython.display import display
from IPython.core.display import HTML
from IPython.display import clear_output

from lib.awws_db_connection.fabula_database_connection import FabulaDatabaseConnection

import lib.utils as utils

In [ ]:
exec(open("../config/encdec.py").read())

In [ ]:
f = open('../config/api-config.json', 'r', encoding='utf-8')
api_config = json.loads(''.join(f.readlines()))

In [ ]:
f = open('./data/crm/train.json', 'r', encoding='utf-8')
train_data = json.loads(''.join(f.readlines()))

In [43]:
len(train_data)

10340

In [44]:
train_data[0]

{'mmid': 1457017,
 'gs': 'ГПШрПЛ02',
 'prod_t': 8708.0,
 'sum': 1114,
 'user_groups': 1,
 'userId': 12.281156926653441,
 'gsid': 'ТСИЗ3201',
 'post_id': '230',
 '__источник': 'фа',
 'материал': 'ТЦШКОЖ14',
 'тираж': '50',
 'длина,м': '0.09',
 'ширина,м': '0.07',
 'способ печати': 'ПЗРШИН11',
 'контурная порезка на рулоне': 'Добавить',
 'макет': 'ТСИЗИММ2'}

Использовать признаки как значения пользовательской формы магазина. \
Из полученной формы собрать запрос в api просчета

In [ ]:
reqs = dict()

ignore_props = {
    'mmid': 1,
    'gs': 1,
#     'prod_t': 1,
    'sum': 1,
    'user_groups': 1,
}

for row in train_data:
    mmid = row['mmid']
    
    reqs[mmid] = reqs.get(mmid, {
        'method': 'calcProductDurations',
        'argument': {
            'postId': '',
            '_firmId': '',
            'mov': {}
        }
    })
    
    for key in row:
        val = row[key]

        if ignore_props.get(key):
            continue

        elif 'prod_t' == key:
            reqs[mmid]['t_prod'] = val
            
        elif 'userId' == key:
            reqs[mmid]['argument']['_firmId'] = custom_dec(val)

        elif 'post_id' == key:
            reqs[mmid]['argument']['postId'] = val

        elif key in ['email', 'tel']:
            continue

        else:
            reqs[mmid]['argument']['mov'][key] = val

Произвести непосредственно запросы. \
10 тыс. запросов (процесс не быстрый)

In [ ]:
url = f"http://{api_config['server']}/api_v1/"

for i, mmid in enumerate(reqs):
    if 't_prod_api' in reqs[mmid]:
        print('pass', i, mmid)
        clear_output()
        continue
    
    body = reqs[mmid]
    res = requests.post(url, json=body);
    resj = res.json()

    t_prod = utils.nget(body, ['t_prod'])
    api_t_prod = utils.nget(resj, ['data', 'производство', 'duration'])

    reqs[mmid]['t_prod_api'] = api_t_prod
        
    print(i, mmid, t_prod, api_t_prod)

    # time.sleep(1)
        
    clear_output()

In [ ]:
final_prod_t_data = []

for row in train_data:
    mmid = row['mmid']
    
    prod_t_api = utils.nget(reqs, [mmid, 't_prod_api'])
    
    if prod_t_api:
        final_prod_t_data.append([mmid, prod_t_api])

In [45]:
final_prod_t_data[0:10]

[[1467211, 622.4847617530847],
 [1469090, 1262.5675675675675],
 [1469108, 1262.5675675675675],
 [1469624, 976.682270901337],
 [1469702, 589.1632653061224],
 [1469758, 1080],
 [1469767, 1620],
 [1470247, 1620],
 [1470274, 1620],
 [1470278, 1620]]

In [ ]:
# Записать в файл

f = open('./data/calc/prod_t.json', 'w', encoding='utf-8')
f.write(json.dumps(final_prod_t_data, ensure_ascii=False))

Получилась json матрица/таблица

```javascript
[
    [идентификатор_записи, рассчитаное_время_работы_из_калькулятора]
]
```